# Deploy machine learning models to Azure

description: (preview) deploy your machine learning or deep learning model as a web service in the Azure cloud.

# Connect to your workspace

In [1]:
!az login
!az account set -s <my subscription>
!az ml workspace list --resource-group=<my resource group>

[
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-canadacentral"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-ncus"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjiaws-uksouth"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-wus"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-cus"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-eus"
  },
  {
    "resourceGroup": "chrjia-rg",
    "subscriptionId": "13e50845-67bc-4ac5-94db-48d493a6d9e8",
    "workspaceName": "chrjia-neurope"
  },
  {


# Register your model

A registered model is a logical container stored in the cloud, containing all files located at `model_path`, which is associated with a version number and other metadata.



## Register a model from a local file

You can register a model by providing the local path of the model. You can provide the path of either a folder or a single file on your local machine.

In [2]:
!wget https://aka.ms/bidaf-9-model -o model.onnx
!az ml model register -n bidaf_onnx -p ./model.onnx

Registering model bidaf_onnx
{
  "cpu": "",
  "createdTime": "2021-04-30T13:09:22.148147+00:00",
  "description": "",
  "experimentName": "",
  "framework": "Custom",
  "frameworkVersion": null,
  "gpu": "",
  "id": "bidaf_onnx:17",
  "memoryInGB": "",
  "name": "bidaf_onnx",
  "properties": "",
  "runId": "",
  "sampleInputDatasetId": "",
  "sampleOutputDatasetId": "",
  "tags": "",
  "version": 17
}


## Deploy your machine learning model

Replace bidaf_onnx:1 with the name of your model and its version number

In [4]:
!az ml model deploy -n myservice -m bidaf_onnx:1 --ic inferenceconfig.json --dc deploymentconfig.json
!az ml service get-logs -n myservice

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Logging into Docker registry chrjiaeastusc6ba5291.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM chrjiaeastusc6ba5291.azurecr.io/azureml/azureml_7610012a4379937742d36a55b47b6388
 ---> 79bfd0273b68
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 385ae6b06287
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjEzZTUwODQ1LTY3YmMtNGFjNS05NGRiLTQ4ZDQ5M2E2ZDllOCIsInJlc291cmNlR3JvdXBOYW1lIjoiY2hyamlhLXJnIiwiYWNjb3VudE5hbWUiOiJjaHJqaWEtZWFzdHVzMmV1YXAyIiwid29ya3NwYWNlSWQiOiI1OTkzZGVjMS01OTczLTQ4NTktOGJlMS03MDczMzJjNTQ5NWQifSwibW9kZWxzIjp7fSwibW9kZWxzSW5mbyI6e319 | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 4851ff31e314
 ---> 1afd10d6ba97
Step 4/5 : RUN mv '/var/azureml-app/tmpjzbj1j5e.py' /var/azureml-app/main.py
 ---> Running in fa6fe4abcaab
 ---> e97b1aa70ffe
Step 5/5 : 

## Call into your model

In [5]:
!curl -v http://localhost:32267
!curl -v -X POST -H "content-type:application/json" -d '{"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}' http://localhost:32267/score

*   Trying 127.0.0.1:32267...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 32267 (#0)
> GET / HTTP/1.1
> Host: localhost:32267
> User-Agent: curl/7.68.0
> Accept: */*
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.10.3 (Ubuntu)
< Date: Fri, 30 Apr 2021 13:14:58 GMT
< Content-Type: text/html; charset=utf-8
< Content-Length: 7
< Connection: keep-alive
< x-ms-request-id: 794f9b76-e9fc-4655-9dcc-980017381685
< 
* Connection #0 to host localhost left intact
HealthyNote: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 127.0.0.1:32267...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 32267 (#0)
> POST /score HTTP/1.1
> Host: localhost:32267
> User-Agent: curl/7.68.0
> Accept: */*
> content-type:application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.10.3 (Ubuntu)
< Date: Fri, 30 Apr 2021 13:14:58 GM

Notice the use of the AZUREML_MODEL_DIR environment variable to locate your registered model. Now that you've added some pip packages, you also need to update your inference configuration with [new configurations](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli#tabpanel_7_azcli) to add in those additional packages

## Deploy again and call your service

Replace `bidaf_onnx:1` with the name of your model and its version number

In [11]:
!az ml model deploy -n myservice_redeploy -m bidaf_onnx:1 --ic inferenceconfig.json --dc re-deploymentconfig.json
!az ml service get-logs -n myservice_redeploy

ACI service creation operation finished, operation "Succeeded"
{
  "computeType": "ACI",
  "environmentDetails": {
    "databricks": {
      "eggLibraries": [],
      "jarLibraries": [],
      "mavenLibraries": [],
      "pypiLibraries": [],
      "rcranLibraries": []
    },
    "docker": {
      "arguments": [],
      "baseDockerfile": null,
      "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
      "baseImageRegistry": {
        "address": null,
        "password": null,
        "registryIdentity": null,
        "username": null
      },
      "enabled": false,
      "platform": {
        "architecture": "amd64",
        "os": "Linux"
      },
      "sharedVolumes": true,
      "shmSize": null
    },
    "environmentVariables": {
      "AZUREML_ENTRY_SCRIPT": "source_dir/echo_score.py",
      "AZUREML_SOURCE_DIRECTORY": "source_dir",
      "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my-deploy-env",
    "python

Then ensure you can send a post request to the service:

In [12]:
!curl -v -X POST -H "content-type:application/json" -d '{"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}' http://localhost:32267/score

Note: Unnecessary use of -X or --request, POST is already inferred.
*   Trying 127.0.0.1:32267...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 32267 (#0)







* upload completely sent off: 94 out of 94 bytes
* Mark bundle as not supporting multiuse









* Connection #0 to host localhost left intact
"test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}"

## Re-deploy to cloud

Once you've confirmed your service works locally and chosen a remote compute target, you are ready to deploy to the cloud.
Change your re-deploy configuration to correspond to the compute target you've chosen, in this case Azure Container Instances.

Deploy your service again

In [13]:
!az ml model deploy -n myservice -m bidaf_onnx:1 --ic inferenceconfig.json --dc re-deploymentconfig.json
!az ml service get-logs -n myservice

ACI service creation operation finished, operation "Succeeded"
{
  "computeType": "ACI",
  "environmentDetails": {
    "databricks": {
      "eggLibraries": [],
      "jarLibraries": [],
      "mavenLibraries": [],
      "pypiLibraries": [],
      "rcranLibraries": []
    },
    "docker": {
      "arguments": [],
      "baseDockerfile": null,
      "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
      "baseImageRegistry": {
        "address": null,
        "password": null,
        "registryIdentity": null,
        "username": null
      },
      "enabled": false,
      "platform": {
        "architecture": "amd64",
        "os": "Linux"
      },
      "sharedVolumes": true,
      "shmSize": null
    },
    "environmentVariables": {
      "AZUREML_ENTRY_SCRIPT": "source_dir/echo_score.py",
      "AZUREML_SOURCE_DIRECTORY": "source_dir",
      "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "my-deploy-env",
    "python

## Call your remote webservice

When you deploy remotely, you may have key authentication enabled. The example below shows how to get your service key with Python in order to make an inference request.

In [15]:
import requests
import json
from azureml.core import Webservice, Workspace

ws = Workspace(subscription_id="13e50845-67bc-4ac5-94db-48d493a6d9e8",
               resource_group="chrjia-rg",
               workspace_name="chrjia-eastus2euap2")

service = Webservice(workspace=ws, name='myservice')
scoring_uri = service.scoring_uri

# If the service is authenticated, set the key or token
key, _ = service.get_keys()

# Set the appropriate headers
headers = {'Content-Type': 'application/json'}
headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response and logs
data = {"query": "What color is the fox", "context": "The quick brown fox jumped over the lazy dog."}
data = json.dumps(data)
resp = requests.post(scoring_uri, data=data, headers=headers)
print(resp.text)

"test is {'query': 'What color is the fox', 'context': 'The quick brown fox jumped over the lazy dog.'}"


In [16]:
print(service.get_logs())

2021-04-30T13:39:36,471832300+00:00 - gunicorn/run 
2021-04-30T13:39:36,485717300+00:00 - rsyslog/run 
2021-04-30T13:39:36,484683000+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_4b37c633d9634b4aa1fd2bd743dc4a42/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-30T13:39:36,486742900+00:00 - iot-server/run 
/usr/sbin/nginx: /azureml-envs/azureml_4b37c633d9634b4aa1fd2bd743dc4a42/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b37c633d9634b4aa1fd2bd743dc4a42/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b37c633d9634b4aa1fd2bd743dc4a42/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_4b37c633d9634b4aa1fd2bd743dc4a42/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC

## Delete resources

In [25]:
!az ml service delete -n myservice
!az ml model delete bidaf_onnx

{
  "id": "bidaf_onnx:17"
}


## Next Steps

Try reading [our documentation](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=python)